In [ ]:
#pip install faiss-cpu sentence-transformers flask
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import pickle
df = pd.read_csv("data.csv")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['question'].tolist())
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings).astype('float32'))
faiss.write_index(index, "index.faiss")
df.to_pickle("data.pkl")
print("index Created ")
#app.py
from flask import Flask, render_template, request
from sentence_transformers import SentenceTransformer
import pandas as pd
import faiss
import numpy as np
import pickle
app = Flask(__name__)
df = pd.read_pickle("data.pkl")
index = faiss.read_index("index.faiss")
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
def search(query):
    q_embed = model.encode([query])
    q_embed = np.array(q_embed).astype('float32')
    D, I = index.search(q_embed, k=1)
    return df.iloc[I[0][0]]
@app.route("/", methods=["GET", "POST"])
def home():
    answer = ""
    if request.method == "POST":
        query = request.form["query"]
        result = search(query)
        answer = result['answer']
    return render_template("index.html", answer=answer)
if __name__ == "__main__":
    app.run(debug=True)
#html
<!DOCTYPE html>
<html>
<head>
    <title>QnA Bot</title>
    <style>
        body { font-family: Arial; margin: 60px; }
        input { width: 60%; padding: 10px; }
        button { padding: 10px 20px; }
        .answer { margin-top: 30px; font-size: 22px; }
    </style>
</head>
<body>
<h2>QnA Chatbot</h2>
<form method="post">
    <input type="text" name="query" placeholder="Ask a question" required>
    <button type="submit">Search</button>
</form>
<div class="answer">{{ answer }}</div>
</body>
</html>

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
